In [30]:
#%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
import pandas as pd
%jsroot on

In [31]:
iOption0 = 0 # piminus, electron

In [32]:
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,95,0,20,20,40,40,60,60,100]
N_centr = int(len(central_bins)/2)
pt_boarders = [0.2,0.3,0.4,0.5,0.6,0.7,0.9,1.1,1.4,1.9,2.5,3.5,5.0]
N_pt = len(pt_boarders)-1
file_path="/home/yoren/bnl/PHENIX/ee/AuAu/"
file_names=["my-50M_piminus_embed_single_new_04-10_v0.root","my-50M_electron_embed_single_new_04-10_v0.root","my-100M_piminus_single_hagedorn_04-5_v0.root"]
file_name=file_names[iOption0]
outfilenames= ['pionminus','electron','pionminus1']#output
filename=outfilenames[iOption0]
part_ids=[9,3]
part_id = part_ids[iOption0]
part_name = ["e^{+}","e^{-}"]
N_start, N_end = 0e6, 50e6

In [33]:
root.gSystem.Load("../AnaTrain/Run14AuAuLeptonComby/MyEvent_C.so")

1

In [34]:
root.gROOT.ProcessLine(
f"TFile *input = new TFile(\"{file_path+file_name}\", \"READ\");\
TTree *T = (TTree *)input->Get(\"tree\");\
MyDileptonAnalysis::MyEvent *myevent = 0;\
TBranch *br = T->GetBranch(\"MyEvent\");\
br->SetAddress(&myevent);\
MyDileptonAnalysis::MyEventContainer *event_container = new MyDileptonAnalysis::MyEventContainer();\
event_container->InitEvent();\
event_container->GetHistsFromFile(\"../../ee_QA/AnaTrain/Run14AuAuLeptonComby/field_map.root\");\
"
); 
from ROOT import myevent, br, event_container

File opened at ../../ee_QA/AnaTrain/Run14AuAuLeptonComby/field_map.root


mcid: 2 - positron, 3 - electron, 8 - pion+, 9 - pion minus

In [35]:
geant3dict = {
  2: "positron",
  3: "electron",
  5: "+muon",
  6: "-muon",
  8: "+pion",
  9: "-pion",
  11: "+kaon",
  12: "-kaon",
  14: "proton",
  15: "antiproton"
}

In [36]:
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0
for ievent in range(N_real_ev):
    if good_event_counter>60: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if not myevent.GetNtrack(): continue
    do_continue = False
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.6 or myevent.GetEntry(ipart).GetN0()>=0:
            do_continue = True
    if not do_continue:
        continue
    good_event_counter+=1
    print(f"Nevent: {ievent}, centrality: {myevent.GetCentrality()}, x: {myevent.GetPreciseX()}, Ntracks: {myevent.GetNtrack()}") 
    event_container.correct_beam_offset()
    for ipart in range(myevent.GetNtrack()):
        print("pt: ", round(myevent.GetEntry(ipart).GetPtPrime(),3),", e/p: ",\
              round(myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot(),3), ", n0: ",\
              myevent.GetEntry(ipart).GetN0(),", disp: ", round(myevent.GetEntry(ipart).GetDisp(),2), ", chi2/npe0: ",\
              round(myevent.GetEntry(ipart).GetChi2()/(myevent.GetEntry(ipart).GetNpe0()+1e-4),2), ", prob: ",\
              round(myevent.GetEntry(ipart).GetProb(),3), ", id: ",  geant3dict[myevent.GetEntry(ipart).GetMcId()])
    for ipart in range(myevent.GetNgentrack()):
        print(myevent.GetGenTrack(ipart).GetPt())

49907886
Nevent: 1, centrality: 3.0, x: 0.3450630009174347, Ntracks: 1
pt:  0.526 , e/p:  0.95 , n0:  2 , disp:  3.93 , chi2/npe0:  16.24 , prob:  0.971 , id:  electron
0.5231568217277527
Nevent: 5, centrality: 82.0, x: 0.3325830101966858, Ntracks: 1
pt:  0.421 , e/p:  0.955 , n0:  2 , disp:  4.21 , chi2/npe0:  7.37 , prob:  0.153 , id:  electron
0.4252394735813141
Nevent: 6, centrality: 57.0, x: 0.336342990398407, Ntracks: 1
pt:  0.416 , e/p:  0.848 , n0:  -9999 , disp:  -9999.0 , chi2/npe0:  1.0 , prob:  0.97 , id:  electron
0.4276764988899231
Nevent: 10, centrality: 64.0, x: 0.33166900277137756, Ntracks: 1
pt:  0.413 , e/p:  0.717 , n0:  2 , disp:  8.16 , chi2/npe0:  20.5 , prob:  0.605 , id:  electron
0.4073379933834076
Nevent: 12, centrality: 17.0, x: 0.33365699648857117, Ntracks: 1
pt:  0.353 , e/p:  0.918 , n0:  5 , disp:  3.08 , chi2/npe0:  9.63 , prob:  0.632 , id:  positron
1.089500069618225
Nevent: 64, centrality: 85.0, x: 0.3437969982624054, Ntracks: 1
pt:  0.485 , e/p:  0.

In [37]:
data_array = []
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0

for ievent in range(int(N_start), min(int(N_end),N_real_ev)):
    if (ievent - N_start) % 1e5 == 0: print("N of events:", ievent - N_start)
    if good_event_counter>100000000: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if not myevent.GetNtrack(): continue
    do_continue = False
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetMcId()==part_id and (myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.5 or myevent.GetEntry(ipart).GetN0()>=0):
            do_continue = True
    if not do_continue:
        continue
    good_event_counter+=1
    event_container.correct_beam_offset()
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetMcId()==part_id and (myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.5 or myevent.GetEntry(ipart).GetN0()>=0):
            data_array.append([int(myevent.GetCentrality()),myevent.GetPreciseZ(),myevent.GetEntry(ipart).GetPtPrime(),\
                           myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot(),myevent.GetEntry(ipart).GetN0(),\
                           myevent.GetEntry(ipart).GetDisp(),myevent.GetEntry(ipart).GetChi2(),myevent.GetEntry(ipart).GetNpe0(),\
                           myevent.GetEntry(ipart).GetProb(),myevent.GetEntry(ipart).GetPhi0Prime()   ])


data = pd.DataFrame(data_array, columns=('centrality','zvertex','pt', 'e/p', 'n0', 'disp', 'chi2', 'npe0', 'prob', 'phi0'))

KeyboardInterrupt: 

In [ ]:
data.head()

,centrality,zvertex,pt,e/p,n0,disp,chi2,npe0,prob,phi0
0,3,2.77458,0.525764,0.950252,2,3.925957,113.663933,7,0.970659,3.116266
1,82,-7.68562,0.421379,0.955185,2,4.207954,66.360344,9,0.152824,0.987925
2,57,-2.12482,0.416442,0.847759,-9999,-9999.000000,-9999.000000,-9999,0.969826,3.971389
3,64,5.98220,0.413126,0.716776,2,8.160769,61.511620,3,0.604905,1.420636
4,85,-3.69534,0.484926,0.696154,3,4.315839,69.104530,3,0.743609,4.017925


In [ ]:
print(data.size, data.shape, good_event_counter)

103760030 (10376003, 10) 10301197


In [ ]:
filepath = '/home/yoren/bnl/PHENIX/ee/ML/'+filename
compression_options = dict(method='zip', archive_name=f'{filename}.csv')

In [ ]:
data.to_csv(f'{filepath}.zip', compression=compression_options, index=False)